Code to deploy Llama3: https://www.youtube.com/@Analyticsvidhya

## Install Dependencies

In [1]:
!pip install -U "transformers==4.40.0" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using c

In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00


## Download Llama 3

In [4]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Query Llama 3

In [5]:
messages = [
    {"role"   : "system",
     "content": "You are a helpful assistant!"},
    {"role"   : "user",
     "content": """What is the meaning of life?"""},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

What a profound and timeless question! The meaning of life is a topic that has been debated and explored by philosophers, scientists, spiritual leaders, and many others throughout history.

While there is no one definitive answer, here are some perspectives that might shed some light on this profound question:

1. **Purpose**: For many people, the meaning of life is to find purpose and fulfillment in their existence. This can be achieved through various means, such as pursuing one's passions, helping others, or making a positive impact in the world.
2. **Existentialism**: From an existentialist perspective, the meaning of life is created by individuals themselves. It's a matter of personal choice and responsibility, where each person must take ownership of their existence and create their own meaning.
3. **Religious and spiritual beliefs**: Many religious and spiritual traditions offer their own answers to this question. For example, some believe that the meaning of life is to worship 

 Just the names seperated by ; and nothing else in your answer!

In [19]:
messages = [
    {"role"   : "system",
     "content": "You are a helpful assistant!"},
    {"role"   : "user",
     "content": """Please name me the types of fallacy that occur in this text.
     These are possible fallacies:
     (Fallacy of Credibility: Abusive, Ad Hominem, Ad Populum, Appeal to False Authority,
     Appeal to Nature, Appeal to Tradition, Guilt by Association, Tu Quoque;
     Fallacy of Logic: Causal Oversimplification, Circular Reasoning, Equivocation, False Analogy False, Causality, False Dilemma,
     Hasty Generalization, Slippery Slope, Straw Man, Fallacy of Division;
     Appeal to Emotion: Appeal to Positive Emotion, Appeal to Anger, Appeal to Fear, Appeal to Pity, Appeal to Ridicule,
     Appeal to Worse Problem)

     Let’s think step by step:
     America is the best place to live, because it's better than any other country
     """},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

What a great text to analyze!

I've identified the following fallacies:

1. **Appeal to Tradition** (Fallacy of Credibility): The statement assumes that America is the best place to live because it's always been considered so. This is an appeal to tradition, which is a fallacious argument.
2. **False Dilemma** (Fallacy of Logic): The statement presents a binary choice, implying that there are only two options: America is the best place to live, or it's not. This oversimplifies the complexity of the issue and ignores other possible options.

These are the fallacies I've identified. Let me know if you'd like me to elaborate or if you have any further questions!


In [27]:
messages = [
    {"role"   : "system",
     "content": "You are a helpful assistant!"},
    {"role"   : "user",
     "content": """
     Identify the types of fallacies present in the given text. Potential fallacies include:

Fallacy of Credibility: Abusive; Ad Hominem; Ad Populum; Appeal to False Authority; Appeal to Nature; Appeal to Tradition; Guilt by Association; Tu Quoque
Fallacy of Logic: Causal Oversimplification; Circular Reasoning; Equivocation; False Analogy; False Causality; False Dilemma; Hasty Generalization; Slippery Slope; Straw Man; Fallacy of Division
Appeal to Emotion: Appeal to Positive Emotion; Appeal to Anger; Appeal to Fear; Appeal to Pity; Appeal to Ridicule; Appeal to Worse Problem
Analyzing the text:

"Every sunrise, the rooster makes a sound. So it is the rooster which makes the sun rise."

Output just the identified fallacies separated by ;.
     """},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

Appeal to False Authority; False Causality; False Analogy
